In [1]:
import openai
import json

In [2]:
client = openai.OpenAI(
    base_url = "https://api.fireworks.ai/inference/v1",
    api_key = "<YOUR_FIREWORKS_API_KEY>"
)

messages = [
    {"role": "system", "content": f"You are a helpful assistant with access to functions. Use them if required."},
    {"role": "user", "content": "What are Nike's net income in 2022?"}
]

# Describe the functions available to the agent in great detail in JSON Schema
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_financial_data",
            "description": "Get financial data for a company given the metric and year.",
            "parameters": {
                "type": "object",
                "properties": {
                    "metric": {
                        "type": "enum",
                        "choices": ["net_income", "revenue", "ebdita"],
                    },
                    "financial_year": {"type": "int", 
                                   "description": "Year for which we want to get financial data."
                    },
                    "company": {
                        "type": "string",
                        "description": "Name of the company for which we want to get financial data."
                    }
                },
                "required": ["metric", "financial_year", "company"],
            },
        },
    }
]

chat_completion = client.chat.completions.create(
    model="accounts/fireworks/models/fw-function-call-34b-v0",
    messages=messages,
    tools=tools,
    temperature=0.1
)
print(chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_GKEtwoiSicLsGwT60Pkd4Xpz",
            "function": {
                "arguments": "{\"metric\": \"net_income\", \"financial_year\": 2022, \"company\": \"Nike\"}",
                "name": "get_financial_data"
            },
            "type": "function",
            "index": 0
        }
    ]
}


In [3]:
# Locally available tool implementation. Can be replaced by any remote API
def get_financial_data(metric: str, financial_year: int, company: str):
    print(f"{metric=} {financial_year=} {company=}")
    if metric == "net_income" and financial_year == 2022 and company == "Nike":
        return {"net_income": 6_046_000_000}
    else:
        raise NotImplementedError()

function_call = chat_completion.choices[0].message.tool_calls[0].function
tool_response = locals()[function_call.name](**json.loads(function_call.arguments))
print(tool_response)

metric='net_income' financial_year=2022 company='Nike'
{'net_income': 6046000000}


In [4]:
agent_response = chat_completion.choices[0].message

# Append the response from the agent
messages.append(
    {
        "role": agent_response.role, 
        "content": "",
        "tool_calls": [
            tool_call.model_dump()
            for tool_call in chat_completion.choices[0].message.tool_calls
        ]
    }
)

# Append the response from the tool 
messages.append(
    {
        "role": "function",
        "content": json.dumps(tool_response)
    }
)

next_chat_completion = client.chat.completions.create(
    model="accounts/fireworks/models/fw-function-call-34b-v0",
    messages=messages,
    tools=tools,
    temperature=0.1
)

print(next_chat_completion.choices[0].message.content)

Nike's net income for the year 2022 was $6,046,000,000. 
